In [26]:
import numpy as np
import networkx as nx
print("Libraries imported")

class GraphSimulation:
    
    def __init__(self, root_dir, loads, functionalities, model, cycles=True):
        self.model = model
        self.cycles = cycles
        self.loads = np.array(loads)
        self.functionalities = np.array(functionalities)
        self.steps = int(np.dot(self.loads, self.functionalities)/2)
        self.A = np.append(np.array([np.ones(loads[0])]).T, np.zeros([loads[0], functionalities[0]]), axis=1)
        self.B = np.append(np.array([np.ones(loads[1])]).T, np.zeros([loads[1], functionalities[1]]), axis=1)
        self.B_fsse_file_path = root_dir + "/data/graph-simulations/p5a_fsse.data"
        self.B_conv_file_path = root_dir + "/data/graph-simulations/p5a_conv.data"
        
    def fsse_factors(self):
        c = np.linspace(1./self.steps, 1, self.steps)
        if(self.model == 'ideal' or self.model=='id'):
            s_a = np.arange(self.functionalities[0], -1, -1)
            s_b = np.arange(self.functionalities[1], -1, -1)
            
            #fig, ax = plt.subplots(1, 2, sharex=True, sharey="row", gridspec_kw={'wspace': 0., 'hspace': 0.}, figsize=(10,5))
            #for i in range(len(s_a)):
            #    ax[0].plot(c, np.linspace(s_a[i], s_a[i], len(c)), label="A_"+str(i))
            #for i in range(len(s_b)):
            #    ax[1].plot(c, np.linspace(s_b[i], s_b[i], len(c)), label="B_"+str(i))
            #ax[0].set_title("monomer A")
            #ax[1].set_title("monomer B")
            #ax[0].set_xlabel("conversion")
            #ax[1].set_xlabel("conversion")
            #ax[0].set_ylabel("fsse factor")
            
            return s_a, s_b
        
        elif(type(self.model) == list):
            s_a = np.arange(self.functionalities[0], -1, -1)
            s_b = np.array(self.model)
            return s_a, s_b
        
        else:
            s_a = np.arange(self.functionalities[0], -1, -1)
            s_b = np.loadtxt(self.B_fsse_file_path)
            c_b = np.loadtxt(self.B_conv_file_path)
            s_b_ = []
            d = 1./self.steps
            for i in range(len(s_b)):
                s_b_.append(np.interp(c, c_b, s_b[i]))
            s_b_ = np.array(s_b_)
            
            #fig, ax = plt.subplots(1, 2, sharey="row", gridspec_kw={'wspace': 0., 'hspace': 0.}, figsize=(10,5))
            #for i in range(len(s_a)):
            #    ax[0].plot(c, np.linspace(s_a[i], s_a[i], len(c)), label="A_"+str(i))
            #for i in range(len(s_b_)):
            #    ax[1].plot(c, s_b_[i], label="B_"+str(i))
            #ax[0].set_title("monomer A")
            #ax[1].set_title("monomer B")
            #ax[0].set_xlabel("conversion")
            #ax[1].set_xlabel("conversion")
            #ax[0].set_ylabel("fsse factor")
            
            return s_a, s_b_
    
    def reaction_pair(self, s_a, s_b):
        r_a, r_b = [], []
        if(len(s_a.shape) == 1):
            r_a = self.A @ s_a
            r_b = self.B @ s_b
            
        R = np.ravel(np.outer(r_a, r_b))
        R = R/np.sum(R)
        i = np.random.choice(len(R), p=R)
        a = int(i/self.loads[1])
        b = i - a * self.loads[1]
        return a, b
    
    def update_monomers(self, a, b):
        for i in range(len(self.A[a])):
            if(self.A[a][i] == 1):
                self.A[a][i] = 0
                self.A[a][i+1] = 1
                break
        for i in range(len(self.B[b])):
            if(self.B[b][i] == 1):
                self.B[b][i] = 0
                self.B[b][i+1] = 1
                break
                
    def is_gel_stockmayer(self, step):
        if(float(step/self.steps) > 1./np.sqrt(max(self.functionalities)-1)):
            return True
        else:
            return False
    
    def run(self, outfile):
        reactions = []
        f = open(outfile, 'w')
        f.write("mmer1\tmmer2\n")
        if(self.model=='id' or self.model=='ideal' or type(self.model) == list):
            s_a, s_b = self.fsse_factors()
        else:
            s_a, s_b = self.fsse_factors()
        
        G = nx.Graph()    
        if not self.cycles:
            G.add_nodes_from(range(sum(self.loads)))
            
        for i in range(int(self.steps)):
            if(self.model == 'id' or self.model == 'ideal' or type(self.model) == list):
                s_b_ = s_b
            else:
                s_b_ = s_b.T[i]
                s_b_ = np.append(s_b_, 0)
                
            if not self.cycles:
                if not self.is_gel_stockmayer(i + 2):
                    same_molecule = True
                    cap = 0
                    a, b = -1, -1
                    while same_molecule:
                        a, b = self.reaction_pair(s_a, s_b_)
                        same_molecule = nx.has_path(G, a, b + self.loads[0])
                        cap += 1
                        if(cap > (self.functionalities[0] * self.loads[0] - i)**2):
                            same_molecule = False
                    G.add_edge(a, b + self.loads[0])
                else:
                    a, b = self.reaction_pair(s_a, s_b_)
            else:
                a, b = self.reaction_pair(s_a, s_b_)
            self.update_monomers(a, b)
            reactions.append([a,b+self.loads[0]])
            f.write(str(a)+"\t"+str(b+self.loads[0])+"\n")
        
        f.close()
        return reactions
    

# ENTER ROOT DIRECTORY HERE 
root_dir = "d:/CAM/PhD_Year2/FSSE/postproc/data/graph-simulations"
if len(root_dir) == 0:
   root_dir = input("Work directory path: ")
     
polymer = "p5a"
if len(polymer) == 0:
   polymer = input("Polymer rootname: ")
   
for i in range(20):
    print(".",end='')
    #gs = GraphSimulation(root_dir, [140,56], [2,5], [5, 3.1, 2.0, 1.5, 1.0, 0])
    gs = GraphSimulation(root_dir, [140,56], [2,5], "fsse", cycles=True)
    gs.B_fsse_file_path = root_dir + f"/{polymer}_fsse.data"
    gs.B_conv_file_path = root_dir + f"/{polymer}_fsse.data"
    i_ = str(i).zfill(2)
    reactions = gs.run(root_dir + f"/data/graph-simulations/{polymer}_graph_{i}.out")



Libraries imported
....................................................................................................
 14.228996753692627 s
